<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250613_BigQuery%E5%9F%BA%E7%A4%8E%E7%AE%A1%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery

In [1]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad251-31-0613-eb999168cd3d.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [2]:
# 定義資料集參數
dataset_id = 'tibame_gad251_31_dataset_py'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad251-31-0613'  # 替換為 Google Cloud 專案 ID
dataset_ref = client.dataset(dataset_id)

# 設定資料集的描述和其他選項
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'This is a new dataset created from Python.'
dataset.location = 'US'  # 設定資料集的地區位置

# 建立資料集
dataset = client.create_dataset(dataset, timeout=30)  # 可設定超時時間
print(f"dataset: {dataset.dataset_id} ,created")


dataset: tibame_gad251_31_dataset_py ,created


In [3]:
# 定義資料集和資料表
dataset_id = 'tibame_gad251_31_dataset_py'
table_id = 'age_table'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# 定義表格參數
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")


table: age_table ,created


In [12]:
# 定義資料集和表資料表名稱
dataset_id = 'tibame_gad251_31_dataset_py'
table_id = 'age_table'

# 定義資料
rows_to_insert = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 35},
    {"name": "Oscar", "age": 29}
]

# 插入資料
errors = client.insert_rows_json(f"{dataset_id}.{table_id}", rows_to_insert)

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")



data insert successfully


In [17]:
# 定義查詢語句
# query = """
#     SELECT name, age
#     FROM `tibame-gad251-31-0613.tibame_gad251_31_dataset_py.age_table`
#     WHERE age > 30
# """

# 使用 f 使用變數的方式
query = f"""
    select name, age
    from `{project_id}.{dataset_id}.{table_id}`
    where age > 30
"""

# 執行查詢
query_job = client.query(query)

# 獲取結果
results = query_job.result()

# 印出結果
for row in results:
    print(f"name: {row.name}, age: {row.age}")

name: Bob, age: 35
name: Bob, age: 35
name: Bob, age: 35


In [ ]:
# 定義資料集和資料表名稱
dataset_id = '資料集ID'
table_id = 'age_table'

# 定義資料表參數
table_ref = client.dataset(dataset_id).table(table_id)

# 刪除資料表
client.delete_table(table_ref)
print(f"table: {table_id} ,deleted")


In [ ]:
# 定義 SQL 查詢以創建模型
query = """
CREATE OR REPLACE MODEL `專案ID.資料集ID.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")


In [ ]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = """
SELECT *
FROM ML.PREDICT(MODEL `專案ID.資料集ID.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)


In [ ]:
# 定義資料集名稱
dataset_id = '資料集ID'

# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")
